In [1]:
# from functools import partial
from pathlib import Path
# from typing import Optional, Tuple
# import cv2
# import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
# from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
# from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import extract_utils as utils
from torch.utils.data import Dataset, DataLoader
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from torch import nn
import torchvision
from scipy.ndimage import affine_transform

## Extract Eigen Vectors

In [2]:
images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/images"
# features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD_1/features/dino_vits16"
features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/"
output_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/eigs_dot1PCA64linearpredlinear_dssubmax_pred2_sg"

In [3]:
# images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/VOC2012/images/trainval/JPEGImages"
# # features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD_1/features/dino_vits16"
# features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/VOC2012/features/dino_vits16"
# output_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/VOC2012/eigs_test/laplacian"
which_matrix= 'laplacian'
which_color_matrix= 'knn'
which_features= 'k'
normalize=True
threshold_at_zero=True
lapnorm= True
K= 5
image_downsample_factor = None
image_color_lambda = 0.0
multiprocessing = 0
batch_size=2
epochs=10

## Incorporating SimSiam

In [4]:
# Define a custom dataset class
class Feature_Dataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __getitem__(self, index):
        return self.features[index]

    def __len__(self):
        return len(self.features)

In [5]:
# class SimSiam(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.projection_head = SimSiamProjectionHead(feats.shape[1], 128,feats.shape[1])
#         self.prediction_head = SimSiamPredictionHead(feats.shape[1], 128, feats.shape[1])
#
#     def forward(self, x):
#         z = self.projection_head(x)
#         p = self.prediction_head(z)
#         z = z.detach()
#         return z, p

In [6]:
# def normalize_affinity_matrix(affinity_matrix, axis=0):
#     if axis == 1:
#         # Normalize by row-wise sums
#         row_sums = torch.sum(affinity_matrix, dim=1, keepdim=True)
#         print(row_sums)
#         normalized_matrix = affinity_matrix / row_sums
#     elif axis == 0:
#         # Normalize by column-wise sums
#         col_sums = torch.sum(affinity_matrix, dim=0, keepdim=True)
#         normalized_matrix = affinity_matrix / col_sums
#     else:
#         raise ValueError("Invalid axis. Axis must be either 0 or 1.")

#     return normalized_matrix

In [7]:
pca_comp=64
pca = PCA(n_components=pca_comp)
utils.make_output_dir(output_dir)
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
for inp in tqdm(inputs):
    index, features_file = inp
    print(index, features_file)
     # Load
    data_dict = torch.load(features_file, map_location='cpu')
    print(data_dict.keys())   #['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape']
    # print("shape=", data_dict['shape'], "k shape", data_dict['k'].shape, "patch_size=", data_dict['patch_size'])
    image_id = data_dict['file'][:-4]
    print(image_id)
    # Load
    output_file = str(Path(output_dir) / f'{image_id}.pth')
    if Path(output_file).is_file():
        print(f'Skipping existing file {str(output_file)}')
        # break
        # return  # skip because already generated

    # Load affinity matrix
    feats = data_dict[which_features].squeeze().cuda()
    # print("Without normalizing, Features Shape is",feats.shape)
    if normalize:
        feats = F.normalize(feats, p=2, dim=-1)
    # print("After normalization, Features Shape",feats.shape)
    # print("which_matrix=", which_matrix)
    # Eigenvectors of affinity matrix
    if which_matrix == 'affinity_torch':
        W = feats @ feats.T
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
            # print("W shape=", W.shape)
        eigenvalues, eigenvectors = torch.eig(W, eigenvectors=True)
        eigenvalues = eigenvalues.cpu()
        eigenvectors = eigenvectors.cpu()
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)


    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity_svd':
        USV = torch.linalg.svd(feats, full_matrices=False)
        eigenvectors = USV[0][:, :K].T.to('cpu', non_blocking=True)
        eigenvalues = USV[1][:K].to('cpu', non_blocking=True)
        print("which matrix=",which_matrix,"eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity':
        # print("Without normalizing, Features Shape is",feats.shape)
        W = (feats @ feats.T)
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
        W = W.cpu().numpy()
        # print("W shape=", W.shape)
        eigenvalues, eigenvectors = eigsh(W, which='LM', k=K)
        eigenvectors = torch.flip(torch.from_numpy(eigenvectors), dims=(-1,)).T
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of matting laplacian matrix
    elif which_matrix in ['matting_laplacian', 'laplacian']:

        # Get sizes
        B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
        if image_downsample_factor is None:
            image_downsample_factor = P
        H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor

        # Upscale features to match the resolution
        if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
            feats = F.interpolate(
                feats.T.reshape(1, -1, H_patch, W_patch),
                size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
            ).reshape(-1, H_pad_lr * W_pad_lr).T

        ### Feature affinities
        # print("Without normalizing, Features Shape is",feats.shape)

        W_feat_ds = (feats @ feats.T)
        # print("shape of w_feat_ds", W_feat_ds.shape)
        max_wfeatds=torch.max(W_feat_ds).item()
        alpha=3
        # print("Before Subtraction")
        count_positive = torch.count_nonzero(torch.greater_equal(W_feat_ds, 0.))
        # print("positive values=", count_positive)
        W_feat_ds_sum = torch.sum(W_feat_ds)
        # print("w_feat_ds_sum=", W_feat_ds_sum)
        W_feat_ds_mean = torch.mean(W_feat_ds)
        # print("Mean of W_feat", W_feat_ds_mean)
        W_feat_ds_median = torch.median(W_feat_ds)
        # print("Median of W_feat", W_feat_ds_median)
        W_feat_ds_std = torch.std(W_feat_ds)
        # print("Standard Deviation of W_feat", W_feat_ds_std)
        W_feat_ds_max = torch.max(W_feat_ds)
        # print("Maximum of W_feat", W_feat_ds_max)
        W_feat_ds_min = torch.min(W_feat_ds)
        # print("Minimum of W_feat", W_feat_ds_min)
        # print("Factor value", (max_wfeatds / alpha))

        W_feat_ds = W_feat_ds - (max_wfeatds / alpha)

        # print("After Subtraction")
        count_positive = torch.count_nonzero(torch.greater_equal(W_feat_ds, 0.))
        # print("positive values=", count_positive)
        W_feat_ds_sum = torch.sum(W_feat_ds)
        # print("w_feat_ds_sum=", W_feat_ds_sum)
        W_feat_ds_mean = torch.mean(W_feat_ds)
        # print("Mean of W_feat", W_feat_ds_mean)
        W_feat_ds_median = torch.median(W_feat_ds)
        # print("Median of W_feat", W_feat_ds_median)
        W_feat_ds_std = torch.std(W_feat_ds)
        # print("Standard Deviation of W_feat", W_feat_ds_std)
        W_feat_ds_max = torch.max(W_feat_ds)
        # print("Maximum of W_feat", W_feat_ds_max)
        W_feat_ds_min = torch.min(W_feat_ds)
        # print("Minimum of W_feat", W_feat_ds_min)
        proj_layer=nn.Linear(pca_comp,pca_comp).cuda()
        pred_layer=nn.Linear(pca_comp,pca_comp).cuda()
        x0=feats

        x0_arr=x0.cpu()
        scale = np.random.uniform(0.8, 1.2)  # Random scaling factor between 0.8 and 1.2
        translation = np.random.uniform(-10, 10, size=2)  # Random translation vector between -10 and 10 in both directions
        rotation = np.random.uniform(-15, 15)  # Random rotation angle between -15 and 15 degrees
        shear = np.random.uniform(-0.2, 0.2, size=2)  # Random shear factor between -0.2 and 0.2 in both directions

        # Define the affine matrix
        affine_matrix = np.array([[scale * np.cos(rotation), -shear[0] * scale * np.sin(rotation), translation[0]],
                                  [shear[1] * scale * np.sin(rotation), scale * np.cos(rotation), translation[1]],
                                  [0, 0, 1]])
        # print(x0_arr.shape)
        x1_arr=affine_transform(x0_arr, affine_matrix)

        z0_arr= pca.fit_transform(x0_arr)
        z1_arr= pca.fit_transform(x1_arr)
        # Define the affine transformation parameters

        # z1_arr=affine_transform(z0_arr, affine_matrix)
#         z1_arr=pca.fit_transform(z1_arr)
        z0 = torch.from_numpy(z0_arr).float()
        z1 = torch.from_numpy(z1_arr).float()

        # feat_list.append(feats)
        feat_dataset_z0 = Feature_Dataset(z0)
        if feats.shape[0]%2==0:
            features_dataloader_z0 = DataLoader(feat_dataset_z0, batch_size=batch_size, shuffle=True)
        else:
            features_dataloader_z0 = DataLoader(feat_dataset_z0, batch_size=batch_size, shuffle=True, drop_last=True)

        feat_dataset_z1 = Feature_Dataset(z1)
        if feats.shape[0]%2==0:
            features_dataloader_z1 = DataLoader(feat_dataset_z1, batch_size=batch_size, shuffle=True)
        else:
            features_dataloader_z1 = DataLoader(feat_dataset_z1, batch_size=batch_size, shuffle=True, drop_last=True)
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
        criterion = NegativeCosineSimilarity()
        proj_optimizer = torch.optim.SGD(proj_layer.parameters(), lr=0.06)
        pred_optimizer = torch.optim.SGD(pred_layer.parameters(), lr=0.06)
        print("Starting Training")
        for epoch in range(epochs):
            total_loss = 0
            for z0_new,z1_new in zip(features_dataloader_z0,features_dataloader_z1):
                z0_new = z0_new.to(device)
                z1_new = z1_new.to(device)
                z0_new=proj_layer(z0_new)
                z1_new=proj_layer(z1_new)
    #             print("z0_new.shape", z0_new.shape)
    #             print("z1_new.shape", z1_new.shape)
                p0=pred_layer(z0_new)
                p1=pred_layer(z1_new)
    #             print("p0.shape", p0.shape)
    #             print("p1.shape", p1.shape)

                # z0_new=z0_new.detach()
                # z1_new=z1_new.detach()
                loss = 0.5 * (criterion(z0_new, p1) + criterion(z1_new, p0))
                total_loss += loss.detach()
                loss.backward()
                proj_optimizer.step()
                pred_optimizer.step()
                proj_optimizer.zero_grad()
                pred_optimizer.zero_grad()
            avg_loss = total_loss / len(features_dataloader_z0)
            print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")
        # z0_projected=proj_layer(z0.to(device))
        projected_feature=pred_layer(z0.to(device))
        print(projected_feature.shape)
        W_feat_siam=torch.matmul(projected_feature, projected_feature.t())
        # max_wfeatsiam=torch.max(W_feat_siam).item()
        # alpha=3
        # W_feat_siam = W_feat_siam - (max_wfeatsiam / alpha)
        # W_feat_siam=torch.matmul(projected_feature[0], projected_feature[0].t())
        W_feat=W_feat_ds + 0.1*W_feat_siam
#         W_feat=normalize_affinity_matrix(W_feat_unnorm, axis=1)
        # print("W_feat.shape=", W_feat.shape)
        # print("W_feat.shape=", W_feat.shape)
        # W_feat=contrastive_affinity(feats, feats.T)
        if threshold_at_zero:
            W_feat = (W_feat * (W_feat > 0))
        W_feat = W_feat / W_feat.max()  # NOTE: If features are normalized, this naturally does nothing
        # W_feat = W_feat.cpu().numpy()
        W_feat = W_feat.detach().cpu().numpy()
        # print("W_feat shape=",W_feat.shape)

        ### Color affinities
        # If we are fusing with color affinites, then load the image and compute
        if image_color_lambda > 0:

            # Load image
            image_file = str(Path(images_root) / f'{image_id}.jpg')
            image_lr = Image.open(image_file).resize((W_pad_lr, H_pad_lr), Image.BILINEAR)
            image_lr = np.array(image_lr) / 255.

            # Color affinities (of type scipy.sparse.csr_matrix)
            if which_color_matrix == 'knn':
                W_lr = utils.knn_affinity(image_lr / 255)
            elif which_color_matrix == 'rw':
                W_lr = utils.rw_affinity(image_lr / 255)

            # Convert to dense numpy array
            W_color = np.array(W_lr.todense().astype(np.float32))
            # print("W_color shape", W_color.shape)

        else:

            # No color affinity
            W_color = 0

        # Combine
        W_comb = W_feat + W_color * image_color_lambda  # combination
        D_comb = np.array(utils.get_diagonal(W_comb).todense())  # is dense or sparse faster? not sure, should check
        # print("W_comb shape= ", W_comb.shape, "D_comb shape",  D_comb.shape)
        if lapnorm:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM', M=D_comb)
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM', M=D_comb)
        else:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM')
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM')
        eigenvalues, eigenvectors = torch.from_numpy(eigenvalues), torch.from_numpy(eigenvectors.T).float()
    print("eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)
    # Sign ambiguity
    for k in range(eigenvectors.shape[0]):
        if 0.5 < torch.mean((eigenvectors[k] > 0).float()).item() < 1.0:  # reverse segment
            eigenvectors[k] = 0 - eigenvectors[k]

    # Save dict
    output_dict = {'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors}
    torch.save(output_dict, output_file)

  0%|          | 0/999 [00:00<?, ?it/s]

0 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/0001_resized.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0001_resized
Starting Training
epoch: 00, loss: -0.95817
epoch: 01, loss: -0.98899
epoch: 02, loss: -0.99269
epoch: 03, loss: -0.99437
epoch: 04, loss: -0.99537
epoch: 05, loss: -0.99605
epoch: 06, loss: -0.99655
epoch: 07, loss: -0.99692
epoch: 08, loss: -0.99721
epoch: 09, loss: -0.99745
torch.Size([1024, 64])


  0%|          | 1/999 [00:24<6:52:46, 24.82s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 1024])
1 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/0002_resized.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0002_resized
Starting Training
epoch: 00, loss: -0.96204
epoch: 01, loss: -0.99019
epoch: 02, loss: -0.99320
epoch: 03, loss: -0.99467
epoch: 04, loss: -0.99557
epoch: 05, loss: -0.99616
epoch: 06, loss: -0.99661
epoch: 07, loss: -0.99696
epoch: 08, loss: -0.99724
epoch: 09, loss: -0.99746
torch.Size([1024, 64])


  0%|          | 2/999 [00:52<7:19:59, 26.48s/it]

...

 97%|█████████▋| 969/999 [6:38:20<11:34, 23.16s/it]

epoch: 09, loss: -0.99794
torch.Size([1024, 64])
eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 1024])
969 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/0971_resized.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0971_resized
Starting Training
epoch: 00, loss: -0.97307
epoch: 01, loss: -0.99333
epoch: 02, loss: -0.99547
epoch: 03, loss: -0.99644
epoch: 04, loss: -0.99702
epoch: 05, loss: -0.99742
epoch: 06, loss: -0.99770
epoch: 07, loss: -0.99792
epoch: 08, loss: -0.99810
epoch: 09, loss: -0.99824
torch.Size([1024, 64])


 97%|█████████▋| 970/999 [6:38:43<11:07, 23.02s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 1024])
970 /home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/0972_resized.pth
dict_keys(['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape'])
0972_resized
Starting Training
epoch: 00, loss: -0.95692
epoch: 01, loss: -0.99015
epoch: 02, loss: -0.99359
epoch: 03, loss: -0.99518
epoch: 04, loss: -0.99610
epoch: 05, loss: -0.99672
epoch: 06, loss: -0.99713
epoch: 07, loss: -0.99746
epoch: 08, loss: -0.99771
epoch: 09, loss: -0.99792
torch.Size([1024, 64])


 97%|█████████▋| 971/999 [6:39:07<10:49, 23.21s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

